<h1> Main.ipynb </h1>
<h2> 1. Import libraries and modules </h2>

In [1]:
import numpy as np
import pandas as pd

from src.Data_Retriever import DataRetrieverNew as DatRet

from src.Optimization.Markowitz_PT import MarkowitzPT as MPT

from src.Optimization.Environment import PortfolioEnvironment as PorEnv
from src.Optimization.RLModelCompilation import RL_Model as RLM
# from src.Optimization.NeuralNet import CustomCNNExtractor 

from src.Result.Menchero_OGA import MencheroOGA as MOGA
from src.Result.IndPortResults import GenerateResult as GR
from src.Result.OverviewResults import ResultConveyor as RC

import time

import warnings
warnings.filterwarnings('ignore')

<h2> 2. Define operating variables </h2>

In [2]:
trading_n = 400
history_usage = 504
n_sectors = 6
n_stocks_per_sector = 4

# For RL algorithm
history_usage_RL = 40
rolling_reward_window = 40

<h2> 3. Define stock dataframe </h2>

In [3]:
ticker_df =  pd.DataFrame({
    "Petroleum": ["EQNR.OL", "SUBC.OL", "BWO.OL"],
    "Food": ["ORK.OL", "MOWI.OL", "LSG.OL"],
    "Materials": ["NHY.OL", "YAR.OL", "RECSI.OL"],
    "Technologies": ["TEL.OL", "NOD.OL", "ATEA.OL"],
    "Financial": ["STB.OL", "DNB.OL", "AKER.OL"],
    "Shipping":["SNI.OL", "BELCO.OL", "ODF.OL"]
})

<h2> 4. Define ESG array </h2>

In [4]:
esg_scores = np.array([36.6, 17.9, 18, 
                18, 23.2, 29.2, 
                15.7, 25.4, 25.6, # Del this
                19.8, 13.8, 18.1, 
                17.3, 14, 12.3, 
                21.2, 26.8, 24.9
                ])

<h2> 5.-7. (Non-necessary for reproducability) </h2>
<h3>Data retrieval</h3>
<h3>Benchmark optimization (MPT)</h3>
<h3>RL optimization x4</h3>
<br>
All of the data has been generated an stored in csvs. Therefore, I can comment out the code lines below.

In [5]:
# Retrieve data from yf API: y-m-d
data = DatRet("2006-07-01", "2025-03-31", ticker_df)
# In function below, set log=True to check for data availability
data.retrieve_data()

YF.download() has changed argument auto_adjust default to True


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

--- Data retrieved successfully ---


In [6]:
# Generate benchmark weights thorugh MPT using Sharpe ratio
# # benchmark = MPT(history_usage, trading_n)
# IMPORTANT: In order to see  the effect of the weights, algo exclude last observation from optimization
# # benchmark.frequency_optimizing()

In [7]:
# objectives = ["Return", "Sharpe", "Sortino", "Sterling", "Return", "Sharpe", "Sortino", "Sterling"]
# esg_compliancy = [True, True, True, True, False, False, False, False]
# # objectives = ["Sharpe"]
# # esg_compliancy = [True]

# for i in range(len(objectives)):
#     reinforcement = RLM(esg_scores, 
#                         objective=objectives[i],
#                         history_usage=history_usage_RL,
#                         rolling_reward_window=rolling_reward_window,
#                         total_timesteps=200,
#                         esg_compliancy=esg_compliancy[i],
#                         )
#     reinforcement.train_model()
#     reinforcement.test_model()

<h2> 8. Attribution analysis </h2>

In [8]:
# paths = ["Return_esg_True", "Sharpe_esg_True",
#          "Sortino_esg_True","Sterling_esg_True",
#          "Return_esg_False", "Sharpe_esg_False",
#          "Sortino_esg_False","Sterling_esg_False",]

# analysis_list = []
# for i in range(len(paths)):
#     att_anal = GR(paths[i],
#             n_sectors, n_stocks_per_sector,
#             trading_n,
#             esg_scores, 
#             ticker_df.columns)
#     att_anal.friple_frequency_analysis()
#     analysis_list.append(att_anal)

<h2> 9. Overview  Result Analysis </h2>

In [9]:
# theta = RC(analysis_list, trading_n)
# theta.convey_results()

<h1> 10. Ad-hoc </h1>

In [10]:
# ret = pd.read_csv("Data/Input/StockReturns.csv")
# weights = pd.read_csv("Data/RL_weights_Sharpe_esg_false.csv")

# train_set = ret.iloc[:int(0.85*len(ret))]
# test_set = ret.iloc[int(0.85*len(ret)):]

In [11]:
# import pandas as pd
# import numpy as np
# import yfinance as yf

In [12]:
start_date = "2006-07-01"
end_date = "2024-03-31"

In [13]:
# class DataRetrieverNew:

#     def __init__(self, start_date, end_date, ticker_df):
#         """
#         doc string
#         """
#         self.start_date = start_date
#         self.end_date = end_date
#         self.ticker_list = np.array(ticker_df).flatten()

#         self.master_daterange = pd.Series(np.repeat(pd.date_range(start=self.start_date, end=self.end_date, freq="B"), 2), name="Date")
#         self.return_df = pd.DataFrame(index=self.master_daterange)
#         self.volume_df = pd.DataFrame(index=self.master_daterange)
#         self.rolling_return_df = pd.DataFrame(index=self.master_daterange)
#         self.rolling_volatility_df = pd.DataFrame(index=self.master_daterange)


#     def z_score(self, arr: np.array):
#         """
#         doc string
#         """
#         z_score_arr = (arr - np.mean(arr)) / np.std(arr)

#         return z_score_arr



#     def retrieve_data(self):
#         """
#         doc string
#         """
#         for i in range(0, len(self.ticker_list), 1):
#             stock_data = yf.download(self.ticker_list[i], start=start_date, end=end_date)

#             date_df = pd.DataFrame(index=self.master_daterange)
#             open = stock_data["Open"]
#             individual_df = pd.merge(date_df, open, on="Date", how="left")
#             inter_df = pd.merge(date_df, stock_data["Close"], on="Date", how="left")
#             individual_df[self.ticker_list[i]].iloc[1::2] = inter_df[self.ticker_list[i]].iloc[::2]
#             return_series = individual_df.interpolate().ffill().bfill()
#             vol_series = pd.merge(date_df, stock_data["Volume"], on="Date", how="left")

#             self.return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]]
#             self.volume_df[self.ticker_list[i]] = self.z_score(vol_series[self.ticker_list[i]])
#             self.rolling_return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]].rolling(40).mean().bfill()
#             self.return_df[self.ticker_list[i]] = return_series[self.ticker_list[i]].rolling(40).std().bfill()
        
#         master_df = pd.concat([self.return_df, self.volume_df, self.rolling_return_df, self.rolling_volatility_df], axis = 1)
#         master_df.to_csv("Mix.csv", index=False)

#         print("--- Data retrieved successfully ---")

In [14]:
# hotel = DataRetrieverNew(start_date,end_date, ["EQNR.OL", "SNI.OL"] )
# hotel.retrieve_data()